In [1]:
# =========================================
# [0] IMPORTS & GLOBAL CONFIG
# =========================================
!pip install optuna-integration[tfkeras]

import os, json
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import optuna
from optuna.pruners import MedianPruner
from pathlib import Path

SEED = 42
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()
os.environ["PYTHONHASHSEED"] = str(SEED)

EPOCHS = 50
NUM_CLASSES = 4
CLASS_NAMES = ["defect", "longberry", "peaberry", "premium"]
IMG_SIZE = (224,224)

# =========================================
# [1] FIND PREPROCESSED ARTIFACTS
# =========================================
CANDIDATE_PATHS = [
    Path("/kaggle/input/coffe-bean-classification-preprocessing/artifacts_preprocess"),
    Path("/kaggle/input/coffee-bean-classification-preprocessing/artifacts_preprocess"),
    Path("/kaggle/input/coffe-bean-classification-preprocessing"),
]
ART_DIR = None
for base in CANDIDATE_PATHS:
    if base.exists():
        possible_dirs = [base] if (base / "split_train.csv").exists() else list(base.rglob("artifacts_preprocess"))
        for p in possible_dirs:
            if (p / "split_train.csv").exists() and (p / "split_val.csv").exists():
                ART_DIR = p
                break
    if ART_DIR: break
if ART_DIR is None:
    input_dir = Path("/kaggle/input")
    for dataset_dir in input_dir.iterdir():
        if dataset_dir.is_dir():
            for p in dataset_dir.rglob("artifacts_preprocess"):
                if (p / "split_train.csv").exists() and (p / "split_val.csv").exists():
                    ART_DIR = p
                    break
        if ART_DIR: break
if ART_DIR is None:
    raise FileNotFoundError("Tidak menemukan artifacts_preprocess")

train_df = pd.read_csv(ART_DIR / "split_train.csv")
val_df   = pd.read_csv(ART_DIR / "split_val.csv")

# =========================================
# [2] TF.DATA PIPELINE
# =========================================
AUTOTUNE = tf.data.AUTOTUNE

def decode_and_resize(image_path, label, target_size):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, target_size, method="bilinear")
    img = tf.cast(img, tf.float32)
    return img, label

def create_dataset(df, target_size, training=True, batch_size=32):
    paths = df["filepath"].values
    labels = df["class_name"].map({c:i for i,c in enumerate(CLASS_NAMES)}).values.astype(np.int32)
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        ds = ds.shuffle(buffer_size=len(df), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(lambda p,l: decode_and_resize(p,l,target_size), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

# =========================================
# [3] MODEL BUILDER: MobileNetV2 Tunable
# =========================================
from tensorflow.keras import layers, models, regularizers

def build_mobilenetv2_tunable(
    dense_units=0,
    dropout=0.2,
    l2=0.0,
    freeze_backbone=True,
    fine_tune_at=None
):
    base = tf.keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=(224,224,3)
    )

    if freeze_backbone:
        base.trainable = False
    else:
        base.trainable = True
        if fine_tune_at is not None:
            for layer in base.layers[:fine_tune_at]:
                layer.trainable = False

    inputs = layers.Input(shape=(224,224,3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)

    if dense_units and dense_units > 0:
        x = layers.Dense(
            dense_units,
            activation="relu",
            kernel_regularizer=regularizers.l2(l2) if l2 > 0 else None
        )(x)

    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)
    return models.Model(inputs, outputs, name="MobileNetV2_Tuned")

# =========================================
# [4] OPTUNA OBJECTIVE
# =========================================
def objective(trial: optuna.Trial):
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 48, 64])
    lr = trial.suggest_float("lr", 1e-5, 3e-3, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    dense_units = trial.suggest_categorical("dense_units", [0, 64, 128, 256])
    l2 = trial.suggest_float("l2", 1e-7, 1e-3, log=True)
    label_smoothing = trial.suggest_float("label_smoothing", 0.0, 0.15)

    freeze_backbone = trial.suggest_categorical("freeze_backbone", [True, False])
    fine_tune_at = None
    if not freeze_backbone:
        # MobileNetV2 layers ~155
        fine_tune_at = trial.suggest_categorical("fine_tune_at", [30, 60, 90, 120])

    ds_train = create_dataset(train_df, target_size=IMG_SIZE, training=True, batch_size=batch_size)
    ds_val   = create_dataset(val_df,   target_size=IMG_SIZE, training=False, batch_size=batch_size)

    model = build_mobilenetv2_tunable(
        dense_units=dense_units,
        dropout=dropout,
        l2=l2,
        freeze_backbone=freeze_backbone,
        fine_tune_at=fine_tune_at
    )

    opt_name = trial.suggest_categorical("optimizer", ["adam", "adamw"])
    if opt_name == "adamw":
        try:
            opt = tf.keras.optimizers.AdamW(
                learning_rate=lr,
                weight_decay=trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
            )
        except Exception:
            opt = tf.keras.optimizers.Adam(learning_rate=lr)
    else:
        opt = tf.keras.optimizers.Adam(learning_rate=lr)

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer=opt, loss=loss_fn, metrics=["accuracy"])

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True, verbose=0),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7, verbose=0),
        optuna.integration.TFKerasPruningCallback(trial, monitor="val_accuracy"),
    ]

    hist = model.fit(ds_train, validation_data=ds_val, epochs=EPOCHS, callbacks=callbacks, verbose=0)
    return float(np.max(hist.history["val_accuracy"]))

# =========================================
# [5] RUN STUDY
# =========================================
study = optuna.create_study(direction="maximize", pruner=MedianPruner(n_startup_trials=5))
study.optimize(objective, n_trials=25, gc_after_trial=True)

print("Best value:", study.best_value)
print("Best params:", study.best_params)

OUTDIR = Path("/kaggle/working/optuna_mobilenetv2")
OUTDIR.mkdir(parents=True, exist_ok=True)

pd.DataFrame(study.trials_dataframe()).to_csv(OUTDIR/"trials.csv", index=False)
with open(OUTDIR/"best_params.json", "w") as f:
    json.dump(study.best_params, f, indent=2)

# =========================================
# [6] RETRAIN BEST + SAVE
# =========================================
best = study.best_params
bs = best["batch_size"]

ds_train = create_dataset(train_df, target_size=IMG_SIZE, training=True, batch_size=bs)
ds_val   = create_dataset(val_df,   target_size=IMG_SIZE, training=False, batch_size=bs)

model = build_mobilenetv2_tunable(
    dense_units=best["dense_units"],
    dropout=best["dropout"],
    l2=best["l2"],
    freeze_backbone=best["freeze_backbone"],
    fine_tune_at=best.get("fine_tune_at", None),
)

if best["optimizer"] == "adamw":
    try:
        opt = tf.keras.optimizers.AdamW(learning_rate=best["lr"], weight_decay=best.get("weight_decay", 1e-5))
    except Exception:
        opt = tf.keras.optimizers.Adam(learning_rate=best["lr"])
else:
    opt = tf.keras.optimizers.Adam(learning_rate=best["lr"])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=opt, loss=loss_fn, metrics=["accuracy"])

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7, verbose=1),
]
hist = model.fit(ds_train, validation_data=ds_val, epochs=EPOCHS, callbacks=callbacks, verbose=1)

model.save(OUTDIR/"best_model.keras")
print("✅ Saved:", OUTDIR/"best_model.keras")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.9 MB/s eta 0:00:00


2026-01-08 02:51:00.756153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767840660.981633      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767840661.048574      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767840661.592436      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767840661.592493      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767840661.592496      24 computation_placer.cc:177] computation placer alr

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


I0000 00:00:1767840695.920659      72 cuda_dnn.cc:529] Loaded cuDNN version 91002
[I 2026-01-08 02:52:58,290] Trial 0 finished with value: 0.7807486653327942 and parameters: {'batch_size': 64, 'lr': 2.653358431419563e-05, 'dropout': 0.0948638495815744, 'dense_units': 128, 'l2': 0.000888902791659179, 'label_smoothing': 0.05903953394380319, 'freeze_backbone': False, 'fine_tune_at': 120, 'optimizer': 'adam'}. Best is trial 0 with value: 0.7807486653327942.
[I 2026-01-08 02:53:50,637] Trial 1 finished with value: 0.8770053386688232 and parameters: {'batch_size': 48, 'lr': 0.0005357543105373378, 'dropout': 0.27355411004194424, 'dense_units': 128, 'l2': 6.34463647226211e-07, 'label_smoothing': 0.08032740794328211, 'freeze_backbone': True, 'optimizer': 'adam'}. Best is trial 1 with value: 0.8770053386688232.
[I 2026-01-08 02:54:29,037] Trial 2 finished with value: 0.8770053386688232 and parameters: {'batch_size': 48, 'lr': 0.000661772731042354, 'dropout': 0.09170794962309503, 'dense_units': 2

Best value: 0.903743326663971
Best params: {'batch_size': 48, 'lr': 0.001134077259790631, 'dropout': 0.3019440423910115, 'dense_units': 256, 'l2': 2.534372517587611e-06, 'label_smoothing': 0.1270763096398634, 'freeze_backbone': True, 'optimizer': 'adam'}
Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 180ms/step - accuracy: 0.4201 - loss: 1.5413 - val_accuracy: 0.7540 - val_loss: 0.6057 - learning_rate: 0.0011
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.7507 - loss: 0.6581 - val_accuracy: 0.8235 - val_loss: 0.4966 - learning_rate: 0.0011
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.8299 - loss: 0.4999 - val_accuracy: 0.8128 - val_loss: 0.4969 - learning_rate: 0.0011
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.8177 - loss: 0.4625 - val_accuracy: 0.8449 - val_loss: 0.4293 - learning_rate: 0.0011
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.8742 - loss: 0.3688 - val_accuracy: 0.8663 - val_loss: 0.3626 - learning_ra